-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://raw.githubusercontent.com/databricks/koalas/master/Koalas-logo.png" width="220"/>
</div>

## Koalas vs. Pandas vs. Spark vs. PySpark

The Koalas project makes data scientists more productive when interacting with big data, by implementing the pandas DataFrame API on top of Apache Spark. By unifying the two ecosystems with a familiar API, Koalas offers a seamless transition between small and large data.

**Goals of this notebook:**
* Demonstrate the similarities of the Koalas API with the pandas API
* Understand the differences in syntax for the same DataFrame operations in Koalas vs PySpark


[Koalas Github](https://github.com/databricks/koalas)

**Requirements:**
* `DBR 6.0 ML`
* `koalas==0.20.0` 

**Data:**
* *[Moro et al., 2014] S. Moro, P. Cortez and P. Rita. A Data-Driven Approach to Predict the Success of Bank Telemarketing. Decision Support Systems, Elsevier, 62:22-31, June 2014*

We will be using the [UCI Machine Learning Repository 
Bank Marketing Data Set](https://archive.ics.uci.edu/ml/datasets/bank+marketing) throughout this demo.

In [3]:
%sh wget https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip -O /tmp/bank.zip

--2020-01-24 07:22:23-- https://archive.ics.uci.edu/ml/machine-learning-databases/00222/bank.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 579043 (565K) [application/x-httpd-php]
Saving to: ‘/tmp/bank.zip’

 0K .......... .......... .......... .......... .......... 8% 186K 3s
 50K .......... .......... .......... .......... .......... 17% 365K 2s
 100K .......... .......... .......... .......... .......... 26% 39.8M 1s
 150K .......... .......... .......... .......... .......... 35% 368K 1s
 200K .......... .......... .......... .......... .......... 44% 32.9M 1s
 250K .......... .......... .......... .......... .......... 53% 41.2M 0s
 300K .......... .......... .......... .......... .......... 61% 40.4M 0s
 350K .......... .......... .......... .......... .......... 70% 40.6M 0s
 400K .......... .......... .......... .......... .......... 79% 375K 0s
 450K .......... .......... .......... .......... .......... 88% 44.4M 0s
 500K .......... .......... .......... .......... .......... 97% 55.9M 0s
 550K .......... ..... 100% 190M=0.7s

2020-01-24 07:22:25 (828 KB/s) - ‘/tmp/bank.zip’ saved [579043/579043]

In [4]:
%sh unzip -o /tmp/bank.zip -d /tmp/bank

Archive: /tmp/bank.zip
 inflating: /tmp/bank/bank-full.csv 
 inflating: /tmp/bank/bank-names.txt 
 inflating: /tmp/bank/bank.csv

In [5]:
file_path = "/bank-full.csv"
dbutils.fs.cp("file:/tmp/bank/bank-full.csv", file_path)
dbutils.fs.head(file_path)

[Truncated to first 65536 bytes]
Out[3]: '"age";"job";"marital";"education";"default";"balance";"housing";"loan";"contact";"day";"month";"duration";"campaign";"pdays";"previous";"poutcome";"y"\n58;"management";"married";"tertiary";"no";2143;"yes";"no";"unknown";5;"may";261;1;-1;0;"unknown";"no"\n44;"technician";"single";"secondary";"no";29;"yes";"no";"unknown";5;"may";151;1;-1;0;"unknown";"no"\n33;"entrepreneur";"married";"secondary";"no";2;"yes";"yes";"unknown";5;"may";76;1;-1;0;"unknown";"no"\n47;"blue-collar";"married";"unknown";"no";1506;"yes";"no";"unknown";5;"may";92;1;-1;0;"unknown";"no"\n33;"unknown";"single";"unknown";"no";1;"no";"no";"unknown";5;"may";198;1;-1;0;"unknown";"no"\n35;"management";"married";"tertiary";"no";231;"yes";"no";"unknown";5;"may";139;1;-1;0;"unknown";"no"\n28;"management";"single";"tertiary";"no";447;"yes";"yes";"unknown";5;"may";217;1;-1;0;"unknown";"no"\n42;"entrepreneur";"divorced";"tertiary";"yes";2;"yes";"no";"unknown";5;"may";380;1;-1;0;"unknown";"no"\n58;"retired";"married";"primary";"no";121;"yes";"no";"unknown";5;"may";50;1;-1;0;"unknown";"no"\n43;"technician";"single";"secondary";"no";593;"yes";"no";"unknown";5;"may";55;1;-1;0;"unknown";"no"\n41;"admin.";"divorced";"secondary";"no";270;"yes";"no";"unknown";5;"may";222;1;-1;0;"unknown";"no"\n29;"admin.";"single";"secondary";"no";390;"yes";"no";"unknown";5;"may";137;1;-1;0;"unknown";"no"\n53;"technician";"married";"secondary";"no";6;"yes";"no";"unknown";5;"may";517;1;-1;0;"unknown";"no"\n58;"technician";"married";"unknown";"no";71;"yes";"no";"unknown";5;"may";71;1;-1;0;"unknown";"no"\n57;"services";"married";"secondary";"no";162;"yes";"no";"unknown";5;"may";174;1;-1;0;"unknown";"no"\n51;"retired";"married";"primary";"no";229;"yes";"no";"unknown";5;"may";353;1;-1;0;"unknown";"no"\n45;"admin.";"single";"unknown";"no";13;"yes";"no";"unknown";5;"may";98;1;-1;0;"unknown";"no"\n57;"blue-collar";"married";"primary";"no";52;"yes";"no";"unknown";5;"may";38;1;-1;0;"unknown";"no"\n60;"retired";"married";"primary";"no";60;"yes";"no";"unknown";5;"may";219;1;-1;0;"unknown";"no"\n33;"services";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";54;1;-1;0;"unknown";"no"\n28;"blue-collar";"married";"secondary";"no";723;"yes";"yes";"unknown";5;"may";262;1;-1;0;"unknown";"no"\n56;"management";"married";"tertiary";"no";779;"yes";"no";"unknown";5;"may";164;1;-1;0;"unknown";"no"\n32;"blue-collar";"single";"primary";"no";23;"yes";"yes";"unknown";5;"may";160;1;-1;0;"unknown";"no"\n25;"services";"married";"secondary";"no";50;"yes";"no";"unknown";5;"may";342;1;-1;0;"unknown";"no"\n40;"retired";"married";"primary";"no";0;"yes";"yes";"unknown";5;"may";181;1;-1;0;"unknown";"no"\n44;"admin.";"married";"secondary";"no";-372;"yes";"no";"unknown";5;"may";172;1;-1;0;"unknown";"no"\n39;"management";"single";"tertiary";"no";255;"yes";"no";"unknown";5;"may";296;1;-1;0;"unknown";"no"\n52;"entrepreneur";"married";"secondary";"no";113;"yes";"yes";"unknown";5;"may";127;1;-1;0;"unknown";"no"\n46;"management";"single";"secondary";"no";-246;"yes";"no";"unknown";5;"may";255;2;-1;0;"unknown";"no"\n36;"technician";"single";"secondary";"no";265;"yes";"yes";"unknown";5;"may";348;1;-1;0;"unknown";"no"\n57;"technician";"married";"secondary";"no";839;"no";"yes";"unknown";5;"may";225;1;-1;0;"unknown";"no"\n49;"management";"married";"tertiary";"no";378;"yes";"no";"unknown";5;"may";230;1;-1;0;"unknown";"no"\n60;"admin.";"married";"secondary";"no";39;"yes";"yes";"unknown";5;"may";208;1;-1;0;"unknown";"no"\n59;"blue-collar";"married";"secondary";"no";0;"yes";"no";"unknown";5;"may";226;1;-1;0;"unknown";"no"\n51;"management";"married";"tertiary";"no";10635;"yes";"no";"unknown";5;"may";336;1;-1;0;"unknown";"no"\n57;"technician";"divorced";"secondary";"no";63;"yes";"no";"unknown";5;"may";242;1;-1;0;"unknown";"no"\n25;"blue-collar";"married";"secondary";"no";-7;"yes";"no";"unknown";5;"may";365;1;-1;0;"unknown";"no"\n53;"technician";"married";"secondary";"no";-3;"no";"no";"unknown";5;"may";166

In [6]:
%fs ls bank-full.csv

path name size dbfs:/bank-full.csv bank-full.csv 4610348

### Loading the dataset as a Spark Dataframe

In [8]:
dbutils.library.installPyPI("koalas")
dbutils.library.restartPython()

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-953331237151221> in <module> 
 ----> 1 dbutils . library . installPyPI ( "koalas" ) 
 2 dbutils . library . restartPython ( ) 

 /local_disk0/tmp/1579850539199-0/dbutils.py in installPyPI (self, project, version, repo, extras) 
 243 def installPyPI ( self , project , version = "" , repo = "" , extras = "" ) : 
 244 return self.print_and_return(self.entry_point.getSharedDriverContext() \
 --> 245 .addIsolatedPyPILibrary(project, version, repo, extras))
 246 
 247 def restartPython ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o336.addIsolatedPyPILibrary.
: org.apache.spark.SparkException: Library utilities are not available on Databricks Runtime for Machine Learning.
	at com.databricks.backend.daemon.driver.SharedDriverContext.getIsolationId(SharedDriverContext.scala:244)
	at com.databricks.backend.daemon.driver.SharedDriverContext.addIsolatedPyPILibrary(SharedDriverContext.scala:544)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:380)
	at py4j.Gateway.invoke(Gateway.java:295)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.GatewayConnection.run(GatewayConnection.java:251)
	at java.lang.Thread.run(Thread.java:748)

In [9]:
df = (spark.read
           .option("inferSchema", "true")
           .option("header", "true")
           .option("delimiter", ";")
           .option("quote", '"')
           .csv("/bank-full.csv"))

display(df)

age job marital education default balance housing loan contact day month duration campaign pdays previous poutcome y 58 management married tertiary no 2143 yes no unknown 5 may 261 1 -1 0 unknown no 44 technician single secondary no 29 yes no unknown 5 may 151 1 -1 0 unknown no 33 entrepreneur married secondary no 2 yes yes unknown 5 may 76 1 -1 0 unknown no 47 blue-collar married unknown no 1506 yes no unknown 5 may 92 1 -1 0 unknown no 33 unknown single unknown no 1 no no unknown 5 may 198 1 -1 0 unknown no 35 management married tertiary no 231 yes no unknown 5 may 139 1 -1 0 unknown no 28 management single tertiary no 447 yes yes unknown 5 may 217 1 -1 0 unknown no 42 entrepreneur divorced tertiary yes 2 yes no unknown 5 may 380 1 -1 0 unknown no 58 retired married primary no 121 yes no unknown 5 may 50 1 -1 0 unknown no 43 technician single secondary no 593 yes no unknown 5 may 55 1 -1 0 unknown no 41 admin. divorced secondary no 270 yes no unknown 5 may 222 1 -1 0 unknown no 29 admin. single secondary no 390 yes no unknown 5 may 137 1 -1 0 unknown no 53 technician married secondary no 6 yes no unknown 5 may 517 1 -1 0 unknown no 58 technician married unknown no 71 yes no unknown 5 may 71 1 -1 0 unknown no 57 services married secondary no 162 yes no unknown 5 may 174 1 -1 0 unknown no 51 retired married primary no 229 yes no unknown 5 may 353 1 -1 0 unknown no 45 admin. single unknown no 13 yes no unknown 5 may 98 1 -1 0 unknown no 57 blue-collar married primary no 52 yes no unknown 5 may 38 1 -1 0 unknown no 60 retired married primary no 60 yes no unknown 5 may 219 1 -1 0 unknown no 33 services married secondary no 0 yes no unknown 5 may 54 1 -1 0 unknown no 28 blue-collar married secondary no 723 yes yes unknown 5 may 262 1 -1 0 unknown no 56 management married tertiary no 779 yes no unknown 5 may 164 1 -1 0 unknown no 32 blue-collar single primary no 23 yes yes unknown 5 may 160 1 -1 0 unknown no 25 services married secondary no 50 yes no unknown 5 may 342 1 -1 0 unknown no 40 retired married primary no 0 yes yes unknown 5 may 181 1 -1 0 unknown no 44 admin. married secondary no -372 yes no unknown 5 may 172 1 -1 0 unknown no 39 management single tertiary no 255 yes no unknown 5 may 296 1 -1 0 unknown no 52 entrepreneur married secondary no 113 yes yes unknown 5 may 127 1 -1 0 unknown no 46 management single secondary no -246 yes no unknown 5 may 255 2 -1 0 unknown no 36 technician single secondary no 265 yes yes unknown 5 may 348 1 -1 0 unknown no 57 technician married secondary no 839 no yes unknown 5 may 225 1 -1 0 unknown no 49 management married tertiary no 378 yes no unknown 5 may 230 1 -1 0 unknown no 60 admin. married secondary no 39 yes yes unknown 5 may 208 1 -1 0 unknown no 59 blue-collar married secondary no 0 yes no unknown 5 may 226 1 -1 0 unknown no 51 management married tertiary no 10635 yes no unknown 5 may 336 1 -1 0 unknown no 57 technician divorced secondary no 63 yes no unknown 5 may 242 1 -1 0 unknown no 25 blue-collar married secondary no -7 yes no unknown 5 may 365 1 -1 0 unknown no 53 technician married secondary no -3 no no unknown 5 may 1666 1 -1 0 unknown no 36 admin. divorced secondary no 506 yes no unknown 5 may 577 1 -1 0 unknown no 37 admin. single secondary no 0 yes no unknown 5 may 137 1 -1 0 unknown no 44 services divorced secondary no 2586 yes no unknown 5 may 160 1 -1 0 unknown no 50 management married secondary no 49 yes no unknown 5 may 180 2 -1 0 unknown no 60 blue-collar married unknown no 104 yes no unknown 5 may 22 1 -1 0 unknown no 54 retired married secondary no 529 yes no unknown 5 may 1492 1 -1 0 unknown no 58 retired married unknown no 96 yes no unknown 5 may 616 1 -1 0 unknown no 36 admin. single primary no -171 yes no unknown 5 may 242 1 -1 0 unknown no 58 self-employed married tertiary no -364 yes no unknown 5 may 355 1 -1 0 unknown no 44 technician married secondary no 0 yes no unknown 5 may 225 2 -1 0 unknown no 55 technician divorced secondary no 0 no no unknown 5 ma

### Loading the dataset as a pandas Dataframe

In [11]:
import pandas as pd

csv_path = "/dbfs/bank-full.csv"

# Read in using pandas read_csv
pdf = pd.read_csv(csv_path, header=0, sep=";", quotechar='"')
display(pdf.head())

age job marital education default balance housing loan contact day month duration campaign pdays previous poutcome y 58 management married tertiary no 2143 yes no unknown 5 may 261 1 -1 0 unknown no 44 technician single secondary no 29 yes no unknown 5 may 151 1 -1 0 unknown no 33 entrepreneur married secondary no 2 yes yes unknown 5 may 76 1 -1 0 unknown no 47 blue-collar married unknown no 1506 yes no unknown 5 may 92 1 -1 0 unknown no 33 unknown single unknown no 1 no no unknown 5 may 198 1 -1 0 unknown no

### Loading the dataset as a Koalas Dataframe

In [13]:
# Import Koalas
import databricks.koalas as ks
import warnings
warnings.filterwarnings("ignore")

# Read in using Koalas read_csv
kdf = ks.read_csv("/bank-full.csv", header=0, sep=";", quotechar='"')

display(kdf.head())

--------------------------------------------------------------------------- 
 ModuleNotFoundError Traceback (most recent call last)
 <command-3967423810138330> in <module> 
 1 # Import Koalas 
 ----> 2 import databricks . koalas as ks
 3 import warnings
 4 warnings . filterwarnings ( "ignore" ) 
 5 

 ModuleNotFoundError : No module named 'databricks'

In [14]:
# Converting to Koalas Dataframe from Spark DataFrame

# Creating a Koalas DataFrame from PySpark DataFrame
# kdf = ks.DataFrame(df)

# # Alternative way of creating a Koalas DataFrame from PySpark DataFrame
# kdf = df.to_koalas()

In [15]:
%SQL

SELECT * FROM 

Note that calling `.head()` in Koalas may not return return the same results as pandas here. Unlike pandas, the data in a Spark dataframe is not ordered - it has no intrinsic notion of index. When asked for the head of a DataFrame, Spark will just take the requested number of rows from a partition. Do not rely on it to return specific rows, instead use `.loc` or `iloc`.

### Indexing Rows

In [18]:
pdf.iloc[3]

In [19]:
# kdf.iloc[3]

Using a scalar integer for row selection is not allowed in Koalas, instead we must supply either a *slice* object or boolean condition.

In [21]:
kdf.iloc[:4]

In [22]:
display(df.limit(4))

In [23]:
# Koalas Dataframe -> PySpark DataFrame
display(kdf.to_spark())

In [24]:
# Getting the number of rows and columns in PySpark
print((df.count(), len(df.columns)))

In [25]:
# Getting the number of rows and columns in Koalas
print(kdf.shape)

### Column Manipulation

Let's have a look at some column operations. Suppose we want to create a new column, where each last contact duration (the `duration` column) is 100 greater than the original duration entry. We will call this new column `duration_new`.

In [28]:
# Creating a column with PySpark
from pyspark.sql.functions import col

df = df.withColumn("duration_new", col("duration") + 100)
display(df)

In [29]:
# Creating a column with Koalas
kdf["duration_new"] = kdf["duration"] + 100
display(kdf.head())

###Filtering

Let's now count the number of instances where `duration_new` is greater than or equal to 300.

In [32]:
# Filtering with PySpark
df_filtered  = df.filter(col("duration_new") >= 300)
print(df_filtered.count())

In [33]:
# Filtering with Koalas
kdf_filtered = kdf[kdf.duration_new >= 300]
print(kdf_filtered.shape[0])

### Value Counts

Suppose we want to have a look at the number of clients for each unique job type.

In [36]:
# To get value counts of the different job types with PySpark
display(df.groupby("job").count().orderBy("count", ascending=False))

In [37]:
# Value counts in Koalas
kdf["job"].value_counts()

###GroupBy

Let's compare group by operations in PySpark versus Koalas. We will create two DataFrames grouped by education, to get the average `age` and maximum `balance` for each education group.

In [40]:
# Get average age per education group using PySpark
df_grouped_1 = (df.groupby("education")
                .agg({"age": "mean"})
                .select("education", col("avg(age)").alias("avg_age")))

display(df_grouped_1)

In [41]:
# Get the maximum balance for each education group using PySpark
df_grouped_2 = (df.groupby("education")
                .agg({"balance": "max"})
                .select("education", col("max(balance)").alias("max_balance")))

display(df_grouped_2)

In [42]:
# Get the average age per education group in Koalas
kdf_grouped_1 = kdf.groupby("education", as_index=False).agg({"age": "mean"})

# Rename our columns
kdf_grouped_1.columns = ["education", "avg_age"]
display(kdf_grouped_1)

In [43]:
# Get the maximum balance for each education group in Koalas
kdf_grouped_2 = kdf.groupby("education", as_index=False).agg({"balance": "max"})
kdf_grouped_2.columns = ["education", "max_balance"]
display(kdf_grouped_2)

### Joins

Let's now look at doing an inner join between our grouped DataFrames, on the `education` attribute.

In [46]:
# Joining the grouped DataFrames on education using PySpark
df_edu_joined = df_grouped_1.join(df_grouped_2, on="education", how="inner")
display(df_edu_joined)

In [47]:
# Joining the grouped DataFrames on education using Koalas
kdf_edu_joined = kdf_grouped_1.merge(kdf_grouped_2, on="education", how="inner")
display(kdf_edu_joined)

### Writing Data

Finally, let's save our joined DataFrames as Parquet files. [Parquet](https://parquet.apache.org/) is an efficient and compact file format to read and write faster.

In [50]:
# Saving the Spark DataFrame as a Parquet file.
spark_out_path = "/dbfs/bank_grouped_pyspark.parquet"

df_edu_joined.write.mode("overwrite").parquet(spark_out_path)

In [51]:
# Saving the Koalas DataFrame as a Parquet file.
koalas_out_path = "/dbfs/bank_grouped_koalas.parquet"

kdf.to_parquet(koalas_out_path, mode="overwrite")